# Module 3 Homework

In [2]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.core.display import HTML

In [3]:
sns.set()

In [4]:
import configparser
config = configparser.ConfigParser()

In [5]:
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [6]:
data_home, output_dir

('/home/cpm6gh/Documents/MSDS/DS5001/data',
 '/home/cpm6gh/Documents/MSDS/DS5001/output')

In [7]:
ngram_order = 3
widx = [f"w{i}" for i in range(ngram_order)]

In [8]:
widx

['w0', 'w1', 'w2']

In [9]:
OHCO = ['chap_num','para_num','sent_num','token_num']

In [10]:
text_file = f"{data_home}/gutenberg/pg42324.txt"

In [11]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [12]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')

In [13]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT",
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]

In [14]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [15]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] - 1

In [16]:
line_a, line_b

(19, 7671)

In [17]:
LINES = LINES.loc[line_a : line_b]

In [18]:
chap_pat = r"^\s*(?:PREFACE|(?:chapter|LETTER)\s+(?:I|X|V|L|C| )+)"


In [19]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False) # Returns a truth vector

In [20]:
LINES.loc[chap_lines]

,line_str
line_num,
271,PREFACE.
343,LETTER I.
467,LETTER II.
594,LETTER III.
636,LETTER IV.
918,CHAPTER I.
1085,CHAPTER II.
1299,CHAPTER III.
1555,CHAPTER IV.


In [21]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

In [22]:
LINES.loc[chap_lines]

,line_str,chap_num
line_num,,
271,PREFACE.,1.0
343,LETTER I.,2.0
467,LETTER II.,3.0
594,LETTER III.,4.0
636,LETTER IV.,5.0
918,CHAPTER I.,6.0
1085,CHAPTER II.,7.0
1299,CHAPTER III.,8.0
1555,CHAPTER IV.,9.0


In [23]:
LINES.chap_num = LINES.chap_num.ffill()

In [24]:
LINES = LINES.dropna(subset=['chap_num']) 

LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')

In [25]:
CHAPS = LINES.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

In [26]:
CHAPS.head(10)

,chap_str
chap_num,
1,\n\nThe event on which this fiction is founded...
2,"\n\n_To Mrs. Saville, England._\n\nSt. Petersb..."
3,"\n\n_To Mrs. Saville, England._\n\nArchangel, ..."
4,"\n\n_To Mrs. Saville, England._\n\nMY DEAR SIS..."
5,"\n\n_To Mrs. Saville, England._\n\nAugust 5th,..."
6,\n\nI am by birth a Genevese; and my family is...
7,\n\nWe were brought up together; there was not...
8,"\n\nWhen I had attained the age of seventeen, ..."
9,"\n\nFrom this day natural philosophy, and part..."


In [27]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [28]:
para_pat = r'\n\n+'

In [29]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [30]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         The event on which this fiction is founded, ha...
         1         I have thus endeavoured to preserve the truth ...
         2         The circumstance on which my story rests was s...
         3         It is a subject also of additional interest to...
         4         The weather, however, suddenly became serene; ...

In [31]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')]

In [32]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         The event on which this fiction is founded, ha...
         1         I have thus endeavoured to preserve the truth ...
         2         The circumstance on which my story rests was s...
         3         It is a subject also of additional interest to...
         4         The weather, however, suddenly became serene; ...

In [33]:
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [34]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')]
SENTS.sent_str = SENTS.sent_str.str.strip()

In [35]:
SENTS.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         The event on which this fiction is founded, ha...
                  1         Darwin, and some of the physiological writers ...
                  2         I shall not be supposed as according the remot...
                  3         yet, in assuming it as the basis of a work of ...
                  4         The event on which the interest of the story d...

In [36]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [37]:
TOKENS.index.names = OHCO[:4]

In [38]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_str'

In [39]:
VOCAB

,term_str,n
term_str,,
0,the,4252
1,and,2993
2,i,2861
3,of,2687
4,to,2123
...,...,...
7034,peeping,1
7035,pyramids,1
7036,towered,1


In [40]:
VOCAB['n_chars'] = VOCAB['term_str'].str.len()
print(VOCAB)
#VOCAB['modified_term_str'] = VOCAB.index
#VOCAB.loc[(VOCAB.n == 1) & (VOCAB.n_chars < 3), 'modified_term_str'] = "<UNK>"

          term_str     n  n_chars
term_str                         
0              the  4252        3
1              and  2993        3
2                i  2861        1
3               of  2687        2
4               to  2123        2
...            ...   ...      ...
7034       peeping     1        7
7035      pyramids     1        8
7036       towered     1        7
7037        bridge     1        6
7038       shelley     1        7

[7039 rows x 3 columns]


In [41]:
#TOKENS['modified_term_str'] = TOKENS.term_str.map(VOCAB.modified_term_str)

In [42]:
def token_to_padded(token, grouper=['sent_num'], term_str='term_str'):
    ohco = token.index.names # We preserve these since they get lost in the shuffle'
    padded = token.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x[term_str]) + ' </s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
    padded.index.names = ohco
    return padded

In [43]:
TOKEN = TOKENS
TOKEN

token_str      term_str
chap_num para_num sent_num token_num                            
1        0        0        0                   The           the
                           1                 event         event
                           2                    on            on
                           3                 which         which
                           4                  this          this
...                                            ...           ...
29       86       0        7          Frankenstein  frankenstein
                           8                    by            by
                           9                  Mary          mary
                           10                    W             w
                  1        0               Shelley       shelley

[76521 rows x 2 columns]

In [44]:
PADDED = token_to_padded(TOKEN, grouper=OHCO[:3], term_str='term_str')
PADDED = PADDED.unstack().reset_index(drop=True).stack().dropna()
PADDED.index.names = ['sent_num', 'token_num']

/tmp/ipykernel_700123/4185945289.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  PADDED = PADDED.unstack().reset_index(drop=True).stack().dropna()


In [45]:
VOCAB2 = PADDED.term_str.value_counts().to_frame('n')

In [46]:
PADDED.head()

term_str
sent_num token_num         
0        0              <s>
         1              the
         2            event
         3               on
         4            which

In [47]:
def padded_to_ngrams(padded, grouper=['sent_num'], n=2):

    ohco = padded.index.names # This gets lost in the grouping
    G = padded.groupby(grouper)
    ng = [None for i in range(ngram_order)]
    for i in range(ngram_order):
        if i == 0:
            ng[0] = G.apply(lambda x: x)
        else:
            ng[i] = G.apply(lambda x: pd.concat([x.shift(i) for i in range(i,-1,-1)], axis=1))
        ng[i].columns = widx[:i+1]    
        ng[i] = ng[i].reset_index(drop=True)
        ng[i].index = padded.index
        ng[i] = ng[i].fillna('</s>')
            
    return ng

In [48]:
NG = padded_to_ngrams(PADDED, n=ngram_order)

In [49]:
sid = 5

In [50]:
NG[0].loc[sid]

,w0
token_num,
0,<s>
1,it
2,was
3,recommended
4,by
5,the
6,novelty
7,of
8,the


In [51]:
NG[1].loc[sid]

,w0,w1
token_num,,
0,</s>,<s>
1,<s>,it
2,it,was
3,was,recommended
4,recommended,by
5,by,the
6,the,novelty
7,novelty,of
8,of,the


In [52]:
NG[2].loc[sid]

,w0,w1,w2
token_num,,,
0,</s>,</s>,<s>
1,</s>,<s>,it
2,<s>,it,was
3,it,was,recommended
4,was,recommended,by
5,recommended,by,the
6,by,the,novelty
7,the,novelty,of
8,novelty,of,the


In [53]:
def ngrams_to_models(ngrams, k=.01):

    model = [None for i in range(ngram_order)]
    K = len(VOCAB2) * k
    
    for i in range(ngram_order):    
        if i == 0:
            model[i] = ngrams[i].value_counts().to_frame('n')
            model[i]['p'] = (model[i].n + k) / (model[i].n.sum() + K)
            model[i]['i'] = np.log2(1/model[i].p)
        else:
            model[i] = ngrams[i].value_counts().to_frame('n')
            model[i]['cp'] = (model[i].n + k) / (model[i-1].n + K)
            model[i]['i'] = np.log2(1/model[i].cp)
        model[i] = model[i].sort_index()
            
    return model

In [54]:
M = ngrams_to_models(NG)

In [55]:
def ngrams_to_matrix_models(ngrams, k=.01):
    M = [None for i in range(ngram_order)]
    for i in range(ngram_order):
        M[i] = NG[i].value_counts()
        if i > 0:
            M[i] = M[i].unstack(fill_value=0).T + k
        else:
            M[i] = M[i].to_frame('i')
        M[i] = M[i] / M[i].sum()
        M[i] = -np.log2(M[i])        
    return M

In [ ]:
MM = ngrams_to_matrix_models(NG)

In [ ]:
MM[2]